In [10]:
# We are gonna create model for the prediction
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

In [29]:
import pickle
with open("preprocessed_Data.pkl", 'rb') as f:
    dataset = pickle.load(f)
dataset_backup = dataset.copy()

In [30]:
dataset.head()

,Region,Soil_Type,Crop,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Weather_Condition,Days_to_Harvest,Yield_tons_per_hectare
0,West,Sandy,Cotton,897.077239,27.676966,False,True,Cloudy,122,6.555816
1,South,Clay,Rice,992.673282,18.026142,True,True,Rainy,140,8.527341
2,North,Loam,Barley,147.998025,29.794042,False,False,Sunny,106,1.127443
3,North,Sandy,Soybean,986.866331,16.644190,False,True,Rainy,146,6.517573
4,South,Silt,Wheat,730.379174,31.620687,True,True,Cloudy,110,7.248251


In [31]:
dataset.columns

Index(['Region', 'Soil_Type', 'Crop', 'Rainfall_mm', 'Temperature_Celsius',
       'Fertilizer_Used', 'Irrigation_Used', 'Weather_Condition',
       'Days_to_Harvest', 'Yield_tons_per_hectare'],
      dtype='object')

In [32]:
# Now there are some string columns like 
# Region, Soil_Type, Crop, Irrigation_Used, Weather_Condition etc
# We are gonna manage them here we didn't manage them in the dataset_preprocessing.ipynb because we have to store the data without outliers
# and we have to use these features when we need to create streamlit app therefore we are going to manage these columns here
dataset['Region'].unique() # there are just 4 values we can use 1, 2, 3, 4 for these respectively so we are going to replace
dataset['Region'].replace({
    "West":1,
    "South":2,
    "North":3,
    "East":4
}, inplace=True)
# Now print hte unique ones
dataset['Region'].unique()

C:\Users\Wakeel\AppData\Local\Temp\ipykernel_10856\359187088.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['Region'].replace({
C:\Users\Wakeel\AppData\Local\Temp\ipykernel_10856\359187088.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['Region'].replace({


array([1, 2, 3, 4])

In [33]:
# Now handling Soil_Type in the same way
dataset['Soil_Type'].unique()
# 'Sandy', 'Clay', 'Loam', 'Silt', 'Peaty', 'Chalky' These are the only types so we also replace them with numbers 
dataset['Soil_Type'].replace({
    "Sandy":1,
    "Clay":2,
    "Loam":3,
    "Silt":4,
    "Peaty":5,
    "Chalky":6
}, inplace=True)
dataset['Soil_Type'].unique()

C:\Users\Wakeel\AppData\Local\Temp\ipykernel_10856\51728041.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['Soil_Type'].replace({
C:\Users\Wakeel\AppData\Local\Temp\ipykernel_10856\51728041.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['Soil_Type'].replace({


array([1, 2, 3, 4, 5, 6])

In [57]:
# Now same with Fertilizer_Used
# dataset['Fertilizer_Used'].unique() # There are only False and True value So we are gonna replace True:1, False:0
# dataset['Fertilizer_Used'].replace({
#     "False":0,
#     "True":1
# }, inplace=True)
# dataset['Fertilizer_Used'].unique() # still this is showing True, False
# dataset['Fertilizer_Used'].info() # Hoon I see iski  type bool ha
dataset['Fertilizer_Used'] = dataset['Fertilizer_Used'].astype(int) # Now it is converted into 0 and 1

In [58]:
dataset.head()

,Region,Soil_Type,Crop,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Weather_Condition,Days_to_Harvest,Yield_tons_per_hectare
0,1,1,1,897.077239,27.676966,0,True,Cloudy,122,6.555816
1,2,2,2,992.673282,18.026142,1,True,Rainy,140,8.527341
2,3,3,3,147.998025,29.794042,0,False,Sunny,106,1.127443
3,3,1,4,986.866331,16.644190,0,True,Rainy,146,6.517573
4,2,4,5,730.379174,31.620687,1,True,Cloudy,110,7.248251


In [59]:
# Now also look at the Crop feature
dataset['Crop'].unique() # These are the crops 'Cotton', 'Rice', 'Barley', 'Soybean', 'Wheat', 'Maize'
dataset.replace({
    'Cotton':1,
    'Rice':2,
    'Barley':3,
    'Soybean':4,
    'Wheat':5,
    'Maize':6
}, inplace=True)
dataset["Crop"].unique()

array([1, 2, 3, 4, 5, 6])

In [61]:
# Same we do for irrigation_Used as we already done abou Fertilizer_Used
dataset['Irrigation_Used'] = dataset['Irrigation_Used'].astype(int)
# dataset['Irrigation_Used'].info

In [62]:
dataset.head()

,Region,Soil_Type,Crop,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Weather_Condition,Days_to_Harvest,Yield_tons_per_hectare
0,1,1,1,897.077239,27.676966,0,1,Cloudy,122,6.555816
1,2,2,2,992.673282,18.026142,1,1,Rainy,140,8.527341
2,3,3,3,147.998025,29.794042,0,0,Sunny,106,1.127443
3,3,1,4,986.866331,16.644190,0,1,Rainy,146,6.517573
4,2,4,5,730.379174,31.620687,1,1,Cloudy,110,7.248251


In [65]:
# Now we are gonna change weather_condiiton
dataset['Weather_Condition'].unique() # 'Cloudy', 'Rainy', 'Sunny'
dataset['Weather_Condition'].replace({
    'Cloudy':1,
    'Rainy':2,
    'Sunny':3
}, inplace=True)
dataset['Weather_Condition'].unique()

array([1, 2, 3])

In [ ]:
# So, now we have done all with our dataset
dataset.head() # all columns are numerical

,Region,Soil_Type,Crop,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Weather_Condition,Days_to_Harvest,Yield_tons_per_hectare
0,1,1,1,897.077239,27.676966,0,1,1,122,6.555816
1,2,2,2,992.673282,18.026142,1,1,2,140,8.527341
2,3,3,3,147.998025,29.794042,0,0,3,106,1.127443
3,3,1,4,986.866331,16.644190,0,1,2,146,6.517573
4,2,4,5,730.379174,31.620687,1,1,1,110,7.248251


In [67]:
# Now we use target variable as Y and the other ones as X
X = dataset.drop('Yield_tons_per_hectare', axis=1)
Y = dataset['Yield_tons_per_hectare']

In [108]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,random_state=14, test_size=0.2)
Model = LinearRegression()
Model.fit(X_train, Y_train)

LinearRegression()

In [109]:
y_pred = Model.predict(X_test)

In [110]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


mse = mean_squared_error(Y_test, y_pred)
mae = mean_absolute_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

print("MSE:", mse)
print("MAE:", mae)
print("R2 Score:", r2)


MSE: 0.24814805953438437
MAE: 0.39752265459265496
R2 Score: 0.913454158724119


In [111]:
import pickle
with open("Model.pkl", 'wb') as f:
    pickle.dump(Model, f)

In [112]:
# Shukar Alhamdulillah half of the work done
# Jupyter Notebook me
import pickle

metrics = {'MSE': mse, 'MAE': mae, 'R2': r2}

with open("metrics.pkl", "wb") as f:
    pickle.dump(metrics, f)
